In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import networkx as nx
import freeman as fm

In [2]:
g = fm.load('marvel.gml')
g.label_nodes('name')
g.set_all_nodes(size=15, labpos='hover')
g.set_all_edges(color=(0, 0, 0, 0.5))

In [3]:
bc = nx.betweenness_centrality(g)

In [4]:
import pandas as pd
data = pd.DataFrame({
    'id' : [i for i in g.nodes],
    'Name': [g.nodes[i]['name'] for i in g.nodes],
    'ConnAmnt': [g.nodes[i]['amount'] for i in g.nodes],
    'Intermediation (r)': [i for i in bc.values()],
})

In [5]:
char_infos = pd.read_csv('../dataset/characters_stats.csv', index_col=0)
semi_df = pd.merge(data, char_infos, how='inner', on=['Name']) 

In [6]:
import numpy as np
semi_df = semi_df.replace('nan', np.nan)
semi_df = semi_df[semi_df['ConnAmnt'].notna()]
semi_df['ConnAmnt'] = pd.to_numeric(semi_df['ConnAmnt'])

In [7]:
semi_df = semi_df.drop_duplicates(subset=['Name'], keep='first')
semi_df = semi_df[semi_df['Alignment'] != 'neutral']
semi_df = semi_df.replace(['good', 'bad'], [0, 1])

In [8]:
semi_df['Intelligence (%)'] = semi_df['Intelligence']/semi_df['Total']
semi_df['Power (%)'] = semi_df['Power']/semi_df['Total']
semi_df['Strength (%)'] = semi_df['Strength']/semi_df['Total']
semi_df['Combat (%)'] = semi_df['Combat']/semi_df['Total']

In [9]:
# Existem muitos personagens nesses dados cuja soma Total é composta de 
# Intelligence  Strength    Speed    Durability   Power   Combat  Total
#     1            1          1         1           0       1       5
semi_df = semi_df[semi_df['Total'] != 5]

In [10]:
import statsmodels.api as sm

# model = sm.OLS(semi_df['Intermediation (r)'], semi_df[['Intelligence', 'Strength', 'Strength (%)', 
#                                                        'Power', 'Power (%)', 'Alignment']])
model = sm.OLS(semi_df['Intermediation (r)'], semi_df[['Intelligence', 'Strength', 'Power', 'Alignment']])
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:     Intermediation (r)   R-squared (uncentered):                   0.351
Model:                            OLS   Adj. R-squared (uncentered):              0.332
Method:                 Least Squares   F-statistic:                              19.05
Date:                Mon, 23 Nov 2020   Prob (F-statistic):                    1.52e-12
Time:                        22:51:38   Log-Likelihood:                          536.16
No. Observations:                 145   AIC:                                     -1064.
Df Residuals:                     141   BIC:                                     -1052.
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intelligence  8.858e-05   2.07e-05      4.285      0.000    4.77e-05       0.000
Strength       7.15e-06   1.69e-05      0.422      0.674   -2.64e-05    4.07e-05
Power        -1.996e-05   2.11e-05     -0.946      0.346   -6.17e-05    2.18e-05
Alignment       -0.0037      0.001     -3.092      0.002      -0.006      -0.001
==============================================================================
Omnibus:                      114.093   Durbin-Watson:                   1.223
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              912.700
Skew:                           2.900   Prob(JB):                    6.45e-199
Kurtosis:                      13.837   Cond. No.                         240.
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [11]:
print(f"Coef:\n{result.params}\n")
print(f"std err:\n{result.bse}\n")
print(f"p-values:\n{result.pvalues}\n")

Coef:
Intelligence    0.000089
Strength        0.000007
Power          -0.000020
Alignment      -0.003723
dtype: float64

std err:
Intelligence    0.000021
Strength        0.000017
Power           0.000021
Alignment       0.001204
dtype: float64

p-values:
Intelligence    0.000034
Strength        0.673722
Power           0.345783
Alignment       0.002398
dtype: float64



In [12]:
# g = fm.load('marvelWeights.gml')
# g.label_nodes('name')
# g.set_all_nodes(size=15, labpos='hover')
# g.set_all_edges(color=(0, 0, 0, 0.5))

# intelligence = {}

# for i in g.nodes:
#     intelligence[i] = float(g.nodes[i]['intelligence'])

# g.scale_nodes_size(intelligence)
# g.draw()

In [13]:
pd.set_option('display.max_rows', semi_df.shape[0]+1)

In [14]:
semi_df['Total_ws'] = semi_df['Total'] - semi_df['Strength']

new = semi_df[['Strength', 'Durability', 'Total_ws']]

In [15]:
new.corr()

,Strength,Durability,Total_ws
Strength,1.000000,0.628778,0.565524
Durability,0.628778,1.000000,0.751434
Total_ws,0.565524,0.751434,1.000000


In [16]:
semi_df.sort_values(by='Intermediation (r)', ascending=True).head(50)

,id,Name,ConnAmnt,Intermediation (r),Alignment,Intelligence,Strength,Speed,Durability,Power,Combat,Total,Intelligence (%),Power (%),Strength (%),Combat (%),Total_ws
156,1011213,Feral,45.0,0.000024,0,38,28,45,28,20,70,229,0.165939,0.087336,0.122271,0.305677,201
170,1009160,Arclight,58.0,0.000031,1,38,63,23,42,52,70,288,0.131944,0.180556,0.218750,0.243056,225
171,1011092,Leech,45.0,0.000032,0,25,5,12,14,62,14,132,0.189394,0.469697,0.037879,0.106061,127
150,1011304,Thundra,45.0,0.000037,0,38,81,32,64,26,54,295,0.128814,0.088136,0.274576,0.183051,214
167,1009322,Goblin Queen,61.0,0.000037,1,75,10,23,28,65,56,257,0.291829,0.252918,0.038911,0.217899,247
141,1009606,Snowbird,58.0,0.000038,0,50,32,27,42,63,52,266,0.187970,0.236842,0.120301,0.195489,234
115,1009360,Hydro-Man,45.0,0.000048,1,38,13,25,80,66,50,272,0.139706,0.242647,0.047794,0.183824,259
102,1009555,Sage,62.0,0.000048,0,75,10,12,14,28,56,195,0.384615,0.143590,0.051282,0.287179,185
70,1009303,Fin Fang Foom,64.0,0.000054,0,50,81,23,100,68,70,392,0.127551,0.173469,0.206633,0.178571,311
139,1010820,Thor Girl,34.0,0.000062,0,75,83,70,84,100,70,482,0.155602,0.207469,0.172199,0.145228,399
